<a href="https://colab.research.google.com/github/ali1810/Big-DATA-week1-/blob/master/pubchem_sol_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Import the required library 
import requests
from bs4 import BeautifulSoup
import re


In [2]:
## Get the SMILES WITH COMPOUND NAME
data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/aspirin/xml")
html = BeautifulSoup(data.content, "xml")
html.find_all("PC-Urn_label")
smiles_tag = html.find(name="PC-Urn_label", string="SMILES")
smiles_parents = smiles_tag.find_parent("PC-InfoData")
smiles = smiles_parents.find('PC-InfoData_value_sval').string
smiles




'CC(=O)OC1=CC=CC=C1C(=O)O'

In [3]:
### code to get the compund solubility based on the compound id which is also called CID of the compound 
x = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
num = 1
sol = []
for i in range(4):
  #print(x%num)
  data=requests.get(x % num)
  
  html = BeautifulSoup(data.content, "xml")
  solubility = html.find(name='TOCHeading', string='Solubility')
  if solubility ==None:
    solub=None
    sol.append(solub)
  else:
    solub=solubility.find_next_sibling('Information').find(name='String').string
    sol.append(solub)
  
  num += 1

print(sol)


[None, None, None, 'greater than or equal to 100 mg/mL at 67.6° F (NTP, 1992)']


In [4]:
data =requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/1983/JSON?heading=Solubility")
#data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/aspirin/xml")
html = BeautifulSoup(data.content, "xml")
html.find_all("PC-Urn_label")



[]

In [5]:
##install pubchem python library
!pip install pubchempy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13834 sha256=4c2044f8941a25447150399a414b9f359a58a8eaf1ca0f225b5123733f760d90
  Stored in directory: /root/.cache/pip/wheels/7c/3d/8c/8192697412e9899dc55bbbb08bbc1197bef333caaa2a71c448
Successfully built pubchempy


In [6]:
import pubchempy as pcp

import pandas as pd

In [9]:
prop=pcp.get_properties([ 'MolecularWeight' 
],'C1=CC=C(C=C1)C=O', 'smiles')


In [10]:
print(prop[0])

{'CID': 240, 'MolecularWeight': '106.12'}


In [11]:
x = list(map(lambda x: x["CID"], prop))

In [12]:
print(x)

[240]


In [13]:
print(x[0])

240


In [ ]:
##install pubchem python library
!pip install pubchempy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13834 sha256=fde94545e6cc6e6d75063ffaa5df42bfb758bfa08c710a01d55e6b640df0d7e0
  Stored in directory: /root/.cache/pip/wheels/7c/3d/8c/8192697412e9899dc55bbbb08bbc1197bef333caaa2a71c448
Successfully built pubchempy


In [59]:
import pubchempy as pcp
import requests
from bs4 import BeautifulSoup
def smiles_to_sol(SMILES):
   prop=pcp.get_properties([ 'MolecularWeight'], SMILES, 'smiles')
   x = list(map(lambda x: x["CID"], prop))
   y=x[0]
   #print(y)
   x = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % y)
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None  
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub

In [27]:
prop=pcp.get_properties([ 'MolecularWeight'], 'CC(=O)OC1=CC=CC=C1C(=O)O','smiles')
x = list(map(lambda x: x["CID"], prop))
y=x[0]
   #print(y)
x = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
data=requests.get(x % y)
html = BeautifulSoup(data.content, "xml")
solubility = html.find(name='TOCHeading', string='Solubility')
print(type(solubility))

<class 'bs4.element.Tag'>


In [16]:

## Taking smiles from user 
smiles = input("Enter your smiles : ")

## passing smiles to a function to get the solubility from Pubchem websites 
smiles_to_sol(smiles)


Enter your smiles : CC(=O)OC1=CC=CC=C1C(=O)O


'less than 1 mg/mL at 73° F (NTP, 1992)'

In [90]:
import pandas as pd
### getting  delaney dataset from disk 
#df=pd.read_csv("/content/drive/MyDrive/delaney.csv")
df=pd.read_excel("/content/drive/MyDrive/pubchem_1_10000.xlsx")

#df

In [81]:
print(len(df))

10000


In [83]:
P_sol=[]
for i in range(len(df)-9500):
  try:
    sol=smiles_to_sol(df.SMILES[i])
    P_sol.append(sol)
  except AttributeError as e:
    sol=='No string'
    P_sol.append(sol)

   # P_sol.append(sol)  



In [84]:
print(len(P_sol))

500


In [91]:
df1 = pd.DataFrame(P_sol)
df1.columns =['Solubility']
print(df1)


                                            Solubility
0                                                 None
1                                                 None
2                                                 None
3    greater than or equal to 100 mg/mL at 67.6° F ...
4                                                 None
..                                                 ...
495                                               None
496                                               None
497                                             5.79 M
498                                               None
499               50 to 100 mg/mL at 64° F (NTP, 1992)

[500 rows x 1 columns]


In [94]:
### merging the data in one dataframe with smiles and solubility 

df_new = pd.concat([df, df1], axis=1)

In [95]:
df_new

,SrN,SMILES,Solubility
0,1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,None
1,2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,None
2,3,C1=CC(C(C(=C1)C(=O)O)O)O,None
3,4,CC(CN)O,greater than or equal to 100 mg/mL at 67.6° F ...
4,5,C(C(=O)COP(=O)(O)O)N,None
...,...,...,...
9995,10591,CC(CCC(=O)NCCS(=O)(=O)O)[C@H]1CC[C@@H]2[C@@]1(...,NaN
9996,10592,CC12CCC(=O)CC1CCC3C2C(CC4(C3CC[C@@]4(C(=O)CO)O...,NaN
9997,10593,CC12CCC(CC1CCC3C2CCC4(C3CC[C@@]4(C(=O)CO)O)C)O,NaN
9998,10594,CC(CCC(=O)NCCS(=O)(=O)O)[C@H]1CC[C@@H]2[C@@]1(...,NaN
